In [2]:
from transformers import pipeline
import torch
import tqdm as notebook_tqdm
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import textwrap
import  matplotlib.pyplot as plt
import warnings
import dask.bag as db
import dask.dataframe as dd
from dask import delayed
import graphviz
from dask import visualize

import dask.multiprocessing
warnings.filterwarnings("ignore")

/opt/python/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-31 02:57:05.318743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 02:57:05.318795: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
!cat /workspaces/Data-Science-Journey/Natural-Language-Processing/Transformers/text_generation/input/robert_frost.txt | head -n5

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 


In [4]:
%%time
review_bag = db.read_text('/workspaces/Data-Science-Journey/Natural-Language-Processing/Transformers/text_generation/input/robert_frost.txt')
review_bag = review_bag.str.rstrip()
review_bag = review_bag.filter(lambda x: len(x) > 0)
lines = review_bag.compute()

CPU times: user 8.18 ms, sys: 8.1 ms, total: 16.3 ms
Wall time: 398 ms


In [5]:
#dask.visualize(review_bag)

In [6]:
lines[0:10]

['Two roads diverged in a yellow wood,',
 'And sorry I could not travel both',
 'And be one traveler, long I stood',
 'And looked down one as far as I could',
 'To where it bent in the undergrowth;',
 'Then took the other, as just as fair,',
 'And having perhaps the better claim',
 'Because it was grassy and wanted wear,',
 'Though as for that the passing there',
 'Had worn them really about the same,']

In [7]:
%%time
lines = [line.rstrip() for line in open('/workspaces/Data-Science-Journey/Natural-Language-Processing/Transformers/text_generation/input/robert_frost.txt')]
lines = [line for line in lines if len(line) > 0]

CPU times: user 1.14 ms, sys: 258 µs, total: 1.4 ms
Wall time: 1.13 ms


In [8]:
gen = pipeline('text-generation')

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
2022-05-31 02:57:26.513295: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-31 02:57:26.513344: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-31 02:57:26.513371: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-d6b1fc): /proc/driver/nvidia/version does not exist
2022-05-31 02:57:26.539398: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
set_seed = 1234

In [10]:
lines[0]

'Two roads diverged in a yellow wood,'

In [11]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gen(lines[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[   {   'generated_text': 'Two roads diverged in a yellow wood, with one of '
                          'the steepest sections coming to a halt.\n'
                          '\n'
                          'Image caption The roads could not be found after '
                          'the accident\n'
                          '\n'
                          "There was one further road leading to the site's "
                          'edge, one at'}]


In [12]:
pp.pprint(gen(lines[0], max_length=20))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[   {   'generated_text': 'Two roads diverged in a yellow wood, and they went '
                          'under the wheels of Mr. Hutton'}]


In [13]:
pp.pprint(gen(lines[0],num_return_sequences=3, max_length=20))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[   {   'generated_text': 'Two roads diverged in a yellow wood, so many people '
                          'had vanished in front of this. The'},
    {   'generated_text': 'Two roads diverged in a yellow wood, where many '
                          'came from, and the highway of our two'},
    {   'generated_text': 'Two roads diverged in a yellow wood, while others '
                          'had the same pattern.\n'
                          '\n'
                          'The other'}]


In [14]:
def wrap(text):
    return textwrap.fill(text, replace_whitespace=False, fix_sentence_endings=True)

In [15]:
out = gen(lines[0], max_length=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
wrap(out[0]['generated_text'])

'Two roads diverged in a yellow wood, with many others looking like the\nsame wood.'